# Spatial Metadata Tutorial

This tutorial demonstrates how to use the `projection.Transform` and `projection.BoundingBox` classes.

## Introduction

The [Transform](https://ghsc.code-pages.usgs.gov/lhp/pfdf/api/projection/transform.html) and [BoundingBox](https://ghsc.code-pages.usgs.gov/lhp/pfdf/api/projection/bbox.html) classes provide information used to spatially locate a raster dataset. In addition to spatial locations, the classes also provide various routines to help facilitate geospatial processing and reprojections. The BoundingBox class is often more useful for geospatial processing, as it provides information on the latitude of a dataset, whereas the Transform class does not. This latitude information is useful when working with angular (geographic) coordinate reference systesms, as the absolute width of longitude units will vary with the latitude of the dataset.

That said, Transform objects provide useful information on pixel resolution and geometries not available from BoundingBox objects. Furthermore, many open-source geospatial libraries rely on affine Transforms to manage raster datasets, so geospatial developers will likely want to use both classes.

Conceptually, Transform and BoundingBox objects both rely on 4 values relating to spatial coordinates. Each also supports an optional CRS, which locates the spatial coordinates on the Earth's surface. The two classes include options to convert distances between CRS units (referred to as "base" units) and explicit metric or imperial units. If you do not provide a CRS, then the object's base units are ambiguous, and the class cannot convert to metric or imperial units. As such, we recommend including CRS information whenever possible.

## Prerequisites

### Install pfdf
To run this tutorial, you must have installed [pfdf 3+ with tutorial resources](https://ghsc.code-pages.usgs.gov/lhp/pfdf/resources/installation.html#tutorials) in your Jupyter kernel. The following line checks this is the case:

In [ ]:
import check_installation

## BoundingBox Objects

You can use `BoundingBox` objects to locate the spatial coordinates of a raster's edges. These objects include a number of methods useful for geospatial manipulations including: locating a raster's center, reprojection, buffering, and identifying UTM zones. You can also convert a BoundingBox to a Transform object when combined with a raster shape.

In [ ]:
from pfdf.projection import BoundingBox

A `BoundingBox` relies on the following 4 properties:

| Property | Description |
| -------- | ----------- |
| `left` | The X coordinate of the box's left edge |
| `bottom` | The Y coordinate of the box's bottom edge |
| `right` | The X coordinate of the box's right edge |
| `top` | The Y coordinate of the box's top edge |

and an optional `crs` property locates these X and Y coordinates on the Earth's surface.

### Create BoundingBox

#### Constructor

You can use the BoundingBox constructor to create a new BoundingBox object. The constructor has four required arguments: `left`, `bottom`, `right`, and `top`. It also accepts an optional `crs` argument:

In [ ]:
# With CRS
BoundingBox(left=-117.95, bottom=34.15, right=-117.85, top=34.20, crs=4326)

In [ ]:
# Without CRS
BoundingBox(left=-117.95, bottom=34.15, right=-117.85, top=34.20)

Note that you may use any CRS - you are not required to use WGS-84. For example, let's create a BoundingBox defined in EPSG:26911, which uses units of meters:

In [ ]:
BoundingBox(left=408022.1201, bottom=3782655.5413, right=415957.1201, top=3789055.5413, crs=26911)

#### from_dict and from_list

Alternatively, you can use the `from_dict` or `from_list` methods to create a BoundingBox from a dict or list/tuple:

In [ ]:
# From a dict. The CRS key is optional
input = {'left': -117.95, 'bottom': 34.15, 'right': -117.85, 'top': 34.20, 'crs': 4326}
BoundingBox.from_dict(input)

In [ ]:
# From a list or tuple. The fifth element (CRS) is optional
BoundingBox.from_list([-117.95, 34.15, -117.85, 34.20, 4326])

Conversely, you can convert a BoundingBox to a dict or list using the `tolist` and `todict` methods:

In [ ]:
bounds = BoundingBox(-117.95, 34.15, -117.85, 34.20)
print(bounds.todict())
print(bounds.tolist())

### Properties

You can return the spatial coordinates of the left, bottom, right, and top edges using properties of the same name:

In [ ]:
bounds = BoundingBox(-121, 30, -119, 40, crs=4326)
print(bounds.left)
print(bounds.bottom)
print(bounds.right)
print(bounds.top)

Alternatively, you can return the X coordinates of the left and right edges using the `xs` property, and the Y coordinates of the top and bottom edges using the `ys` properties:

In [ ]:
print(bounds.xs)
print(bounds.ys)

You can also return the (X, Y) coordinate of the box's center using the `center` property:

In [ ]:
bounds.center

The `crs` property returns the box's CRS as a [pyproj.CRS object](https://pyproj4.github.io/pyproj/stable/examples.html):

In [ ]:
bounds.crs

and `units` returns the units of the CRS along the X and Y axes:

In [ ]:
bounds.units

And you can use `units_per_m` to convert these units to meters:

In [ ]:
bounds.units_per_m

For angular (geographic) coordinate systems, the number of X units per meter will depend on the latitude of the dataset because longitude units become shorter at higher latitudes. Here, the reported X units per meter is specifically the value at the center of the BoundingBox.

Finally, the `orientation` property returns the Cartesian quadrant that would contain the box if the origin point were defined as the box's minimum (X, Y) coordinate. Equivalently, the orientation indicates whether left <= right, and whether bottom <= top. For example:

In [ ]:
BoundingBox(0, 2, 10, 5).orientation

In [ ]:
BoundingBox(10, 2, 0, 5).orientation

In [ ]:
BoundingBox(10, 5, 0, 2).orientation

In [ ]:
BoundingBox(0, 5, 10, 2).orientation

### Height and Width

Use the `height` method to return the distance between the top and bottom edges of the BoundingBox. Similarly, use `width` to return the distance between left and right. By default, these methods return values in CRS base units, but you can use the `units` option to return values in other units instead:

In [ ]:
# CRS base units
bounds = BoundingBox(-121, 30, -119, 35, crs=4326)
print(bounds.height())
print(bounds.width())

In [ ]:
# In kilometers
print(bounds.height('kilometers'))
print(bounds.width('kilometers'))

#### Note
The `height` and `width` methods always return positive values. If orientation is important, you can alternatively use `xdisp` to return (right - left) and `ydisp` to return (top - bottom). These two values may be negative, depending on the orientation of the box.

### Reprojection
BoundingBox objects provide several methods to support CRS reprojection. The `utm_zone` method returns the CRS of the UTM zone overlapping the box's center:

In [ ]:
bounds = BoundingBox(-121, 30, -119, 35, crs=4326)
bounds.utm_zone()

and the `reproject` method returns a copy of the box reprojected to the indicated CRS:

In [ ]:
bounds = BoundingBox(-121, 30, -119, 35, crs=4326)
bounds.reproject(crs=26911)

Two convenience methods provide quick reprojection to common CRSs. The `to_utm` method reprojects the box to the UTM zone overlapping the center point, and `to_4326` reprojects the box to EPSG:4326 (often referred to as WGS 84):

In [ ]:
bounds = BoundingBox(1.1e5, 3.3e6, 3.1e5, 3.8e6, crs=26911)
print(bounds.to_utm())
print(bounds.to_4326())

### Misc
You can use the `orient` method to return a copy of the BoundingBox in the requested orientation. By default, this method places the box in the first Cartesian quadrant, but you can optionally specify a different quadrant instead:

In [ ]:
# Reorient into the first quadrant
bounds = BoundingBox(100, 8, 50, 1)
bounds.orient()

In [ ]:
# Or other quadrants
print(bounds.orient(2))
print(bounds.orient(3))
print(bounds.orient(4))

Separately, you can use the `buffer` method to return a copy of the box that has been buffered by a specified distance:

In [ ]:
# Buffer all edges the same amount
bounds = BoundingBox(50, 0, 2000, 4000, crs=26911)
bounds.buffer(2, units='kilometers')

In [ ]:
# Buffer edges by specific distances
bounds.buffer(left=0, right=12, bottom=100, top=50)

### Transform Conversion
When combined with a raster shape, a BoundingBox can be converted to a Transform object. This can be useful if you need to determine resolution or pixel geometries for the raster. To convert a BoundingBox object, use the `transform` method with a raster shape:

In [ ]:
# BoundingBox and raster shape
bounds = BoundingBox(50, 0, 2000, 4000, crs=26911)
nrows, ncols = (1000, 200)

# Convert to Transform
bounds.transform(nrows, ncols)

## Transform Objects

You can use `Transform objects` to describe a raster's affine transformation matrix. These objects include a number of methods with information on pixel geometries and resolution. You can also convert a Transform to a BoundingBox object when combined with a raster shape.

In [ ]:
from pfdf.projection import Transform

A Transform relies on the following 4 values:

| Property | Description |
| -------- | ----------- |
| `dx` | The change in X coordinate when moving one pixel right |
| `dy` | The change in Y coordinate when moving one pixel down |
| `left` | The X coordinate of the left edge of the raster |
| `top` | The Y coordinate of the top edge of the raster |

and an optional `crs` property determines the location of X and Y coordinates on the Earth's surface.

### Create Transform
You can use the Transform constructor to create a new Transform object. The constructor has four required arguments: `dx`, `dy`, `left`, and `top` and an optional `crs` argument:

In [ ]:
# With CRS
Transform(10, -10, 5000, 19, crs=26911)

In [ ]:
# Without CRS
Transform(dx=10, dy=-10, left=5000, top=19)

Alternatively, you can use the `from_dict`, `from_list`, and `from_affine` commands to create a Transform from a dict, list, tuple, or [affine.Affine object](https://pypi.org/project/affine/):

In [ ]:
# From a dict. CRS key is optional
input = {'dx': 10, 'dy': -10, 'left': 5000, 'top': 19, 'crs': 26911}
Transform.from_dict(input)

In [ ]:
# From a list or tuple. Fifth element (CRS) is optional
Transform.from_list([10, -10, 5000, 19, 26911])

In [ ]:
# From an affine.Affine object
from affine import Affine
input = Affine(10, 0, 5000, 0, -10, 19)
Transform.from_affine(input)

Conversely, you can convert a Transform to a dict or list using the `tolist` and `todict` methods:

In [ ]:
transform = Transform(10, -10, 5000, 19)
print(transform.todict())
print(transform.tolist())

### Properties
You can return left and top using properties of the same name:

In [ ]:
print(transform.left)
print(transform.top)

and `crs` returns the CRS as a [pyproj.crs](https://pyproj4.github.io/pyproj/stable/examples.html):

In [ ]:
transform = Transform(10, -10, 5000, 19, 26911)
transform.crs

You can also query the base units of the CRS (along the X and Y axes) using the `units` property:

In [ ]:
transform.units

The `affine` property returns the Transform as an `affine.Affine` object suitable for coordinate mathematics:

In [ ]:
transform.affine

and `orientation` returns the Cartesian quadrant that would contain the raster if the origin point were defined using the raster's minimum X and Y coordinates. Equivalently, the quadrant is determined by the sign of the `dx` and `dy` values:

In [ ]:
Transform(1,-1,0,0).orientation

In [ ]:
Transform(-1,-1,0,0).orientation

In [ ]:
Transform(-1,1,0,0).orientation

In [ ]:
Transform(1,1,0,0).orientation

### Orientation
You can return `dx`, `dy`, and a tuple of (X axis, Y axis) resolution using the methods of the same name:

In [ ]:
transform = Transform(10, -10, 0, 0, 26911)
print(transform.dx())
print(transform.dy())
print(transform.resolution())

Note that resolution is the absolute value of dx and dy, so is strictly positive. By default, these methods will return values in the base unit of the CRS, and you can use the `units` option to return the values in explicit metric or imperial units:

In [ ]:
# Default is CRS base units
transform = Transform(9e-5, 9e-5, -121, 0, 4326)
print(transform.dx())
print(transform.dy())

In [ ]:
# Select other units
print(transform.dx(units="meters"))
print(transform.dy("feet"))

The values for `dy` are always constant. However, `dx` values are variable when using an angular (geographic) CRS, due to the changing width of longitude units at different latitudes. By default, `dx` and `resolution` return values as measured at the equator. However, you can use the `y` input to obtain more accurate results at other latitudes. This input should be the latitude of the raster's center in the base units of the angular CRS. In practice, this is typically units of decimal degrees:

In [ ]:
# Values measured at the equator
transform = Transform(9e-5, -9e-5, -121, 30, crs=4326)
print(transform.dx("meters"))
print(transform.resolution("meters"))

In [ ]:
# dx is smaller at higher latitudes
print(transform.dx("meters", y=35))
print(transform.resolution("meters", y=35))

### Pixel Geometries
You can use the `pixel_area` method to return the area of a single pixel, and `pixel_diagonal` to return the length of a pixel diagonal. Both of these commands support the `units` and `y` options discussed in the previous section:

In [ ]:
transform = Transform(9e-5, -9e-5, -121, 30, 4326)
print(transform.pixel_area("meters"))
print(transform.pixel_area("meters", y=35))

In [ ]:
print(transform.pixel_diagonal("meters"))
print(transform.pixel_diagonal("meters", y=35))

### BoundingBox Conversion
When combined with a raster shape, a Transform can be converted to a BoundingBox object. This can be useful, as BoundingBox objects include methods not supported by Transform objects. For example, you can use a BoundingBox to return the raster's center, determine the best UTM projection, or determine the bounds of a buffered raster.

To convert a Transform object, use the `bounds` method with a raster shape:

In [ ]:
# Transform object and raster shape
transform = Transform(10, -10, 0, 0, 26911)
nrows, ncols = (1000, 2000)

# Convert to BoundingBox
transform.bounds(nrows, ncols)

### Reprojection
Transform objects include a `reproject` method, which will convert the Transform to a different CRS:

In [ ]:
transform = Transform(10, -10, 0, 0, 26911)
transform.reproject(crs=4326)

#### Important!
BoundingBox objects provide more accurate reprojections than Transform objects. As such, the preferred reprojection workflow for a Transform is as follows:

1. Convert the Transform to a BoundingBox object,
2. Reproject the BoundingBox,
3. Convert the reprojected BoundingBox back to a Transform